In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
#!pip install keras
from IPython import display
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

from collections import Counter
from matplotlib.lines import Line2D

data = pd.read_excel('data/recruitment.xls', index_col=False)
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dropout, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping

In [2]:
#ALL CODE IS HEAVILY USED AND MINORLY EDITED FOR THE GIVEN DATA
#MAJORITY OF CODE DONE BY:
#https://github.com/RaffaeleAns/Adversarial-Fair-Classifier/blob/master/code.ipynb

In [3]:
data.head()

,ApplicantCode,Gender,BAMEyn,ShortlistedNY,Interviewed,FemaleONpanel,OfferNY,AcceptNY,JoinYN
0,1,1,2,1,1,1.0,1.0,1.0,1.0
1,2,1,2,1,1,1.0,1.0,1.0,1.0
2,3,1,2,1,1,1.0,1.0,1.0,1.0
3,4,1,2,1,1,2.0,1.0,1.0,1.0
4,5,1,2,1,1,2.0,0.0,NaN,NaN


In [4]:
data.replace(np.nan,0, inplace=True)
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
lbl = LabelBinarizer()
data["Gender"] = lbl.fit_transform(data["Gender"]).flatten()
data["BAMEyn"] = lbl.fit_transform(data["BAMEyn"]).flatten()
X = data.drop(["Gender","BAMEyn","ApplicantCode","OfferNY","AcceptNY","JoinYN"], axis=1)
Y = data["OfferNY"]
Z = data[["Gender","BAMEyn"]]

In [5]:
X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, Y, Z, test_size=0.3, 
                                                                     stratify=Y, random_state=7)
# split into train/val set
X_train, X_val, y_train, y_val, Z_train, Z_val = train_test_split(X_train, y_train, Z_train, test_size=0.3, 
                                                                     stratify=y_train, random_state=7)

In [6]:
# Converting to nparray
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [7]:
# Scaling
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [8]:
#use logistic regression classifier
def lr_classifier(inputs):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
    return model

In [9]:
def compute_class_weights(df):
    class_values = [0, 1]
    class_weights = []

    if len(df.shape) == 1:
        balanced_weights = compute_class_weight(class_weight = "balanced", classes=class_values, y =df)
        class_weights.append(dict(zip(class_values, balanced_weights)))

    else:
        n_attr =  df.shape[1]

        for idx in range(n_attr):
            balanced_weights = compute_class_weight(class_weight = "balanced", classes=class_values,
                                                    y = np.array(df)[:,idx])
            class_weights = [(dict(zip(class_values, balanced_weights)))]

    return class_weights

In [10]:
# initialise Logistic Regression without sklearn
n_features=X_train.shape[1]
classifier_inputs = Input(shape=(n_features,), name="clf_input")

clf = lr_classifier(classifier_inputs)
class_weights_clf = compute_class_weights(y_train)
# train on train set
history = clf.fit(X_train, y_train, 
                  epochs=50, 
                  validation_data=(X_val, y_val),
                  verbose=1,
                  class_weight=class_weights_clf[0])

Epoch 1/50
5/5 [==============================] - 1s 91ms/step - loss: 1.7412 - accuracy: 0.1022 - val_loss: 2.3522 - val_accuracy: 0.1017
Epoch 2/50
5/5 [==============================] - 0s 43ms/step - loss: 1.6673 - accuracy: 0.1022 - val_loss: 2.3213 - val_accuracy: 0.1017
Epoch 3/50
5/5 [==============================] - 0s 7ms/step - loss: 1.6143 - accuracy: 0.1022 - val_loss: 2.2915 - val_accuracy: 0.1017
Epoch 4/50
5/5 [==============================] - 0s 7ms/step - loss: 1.5702 - accuracy: 0.1022 - val_loss: 2.2639 - val_accuracy: 0.1017
Epoch 5/50
5/5 [==============================] - 0s 7ms/step - loss: 1.5329 - accuracy: 0.1022 - val_loss: 2.2362 - val_accuracy: 0.1017
Epoch 6/50
5/5 [==============================] - 0s 7ms/step - loss: 1.4974 - accuracy: 0.1022 - val_loss: 2.2101 - val_accuracy: 0.1017
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 1.4621 - accuracy: 0.1022 - val_loss: 2.1857 - val_accuracy: 0.1017
Epoch 8/50
5/5 [================

In [11]:
# predict on test set
probs_b = clf.predict(X_test)
#fairness metric
def p_rule(y_pred, z_values, threshold=0.5):
    y_z_1 = y_pred[z_values == 1] > threshold if threshold else y_pred[z_values == 1]
    y_z_0 = y_pred[z_values == 0] > threshold if threshold else y_pred[z_values == 0]
    odds = y_z_1.mean() / y_z_0.mean()
    return np.min([odds, 1/odds]) *100

In [12]:
#make data frames for all values
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)
Z_train = pd.DataFrame(Z_train)

X_val = pd.DataFrame(X_val)
y_val = pd.DataFrame(y_val)
Z_val = pd.DataFrame(Z_val)

X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)
Z_test = pd.DataFrame(Z_test)
#Fuction to make hidden layers trainable
def _make_trainable(net):
    def make_trainable(flag):
        net.trainable = flag
        for layer in net.layers:
            layer.trainable = flag
    return make_trainable

In [13]:
#Classifier using neural net
def build_classifier(inputs):
    dense1 = Dense(32, activation='relu', name="dense_1")(inputs)
    dropout1 = Dropout(0.2, name="dropout_1")(dense1)
    dense2 = Dense(32, activation='relu', name="dense_2")(dropout1)
    dropout2 = Dropout(0.2, name="dropout_2")(dense2)
    dense3 = Dense(32, activation='relu', name="dense_3")(dropout2)
    dropout3 = Dropout(0.2, name="dropout_3")(dense3)
    outputs = Dense(1, activation='sigmoid', name='y')(dropout3)
    return Model(inputs=[inputs], outputs=[outputs], name="classifier")

In [14]:
#Adversial neural net
def build_adversial(inputs, n_sensitive):
    dense1 = Dense(32, activation='relu', name="dense_1")(inputs)
    dense2 = Dense(32, activation='relu', name="dense_2")(dense1)
    dense3 = Dense(32, activation='relu', name="dense_3")(dense2)
    outputs = [Dense(1, activation='sigmoid', name="adv_output_{:>01}".format(i))(dense3) for i in range(n_sensitive)]
    return Model(inputs=[inputs], outputs=outputs, name="adversial")

In [15]:
#Compile classifier
def compile_classifier(classifier):
    trainable_classifier = _make_trainable(classifier)
    trainable_classifier(True)
    classifier.compile(loss='binary_crossentropy', optimizer=Adam())
    return classifier

In [16]:
#Compile Classification - Adversial Net
def compile_can(inputs, classifier, adversial, lambda_0, lambda_1):
    #lambdas= [lambda_0, lambda_1]
    can = Model(inputs=[inputs], outputs=[classifier(inputs)]+adversial(classifier(inputs)), name="can")
    trainable_classifier = _make_trainable(classifier)
    trainable_adversial = _make_trainable(adversial)
    trainable_classifier(True)
    trainable_adversial(False)
    loss_weights = [1.]+[-lambda_0]+[-lambda_1]
    can.compile(loss=['binary_crossentropy']*(len(loss_weights)), 
                      loss_weights=loss_weights,
                      optimizer=Adam())
    return can

In [17]:
#Compile Adversial Net (same as Classification-Adversial but with only two outputs corresponding to the sensitive attributes)
def compile_adversial(inputs, classifier, adversial, n_sensitive):
    adv = Model(inputs=[inputs], outputs=adversial(classifier(inputs)))
    trainable_classifier = _make_trainable(classifier)
    trainable_adversial = _make_trainable(adversial)
    trainable_classifier(False)
    trainable_adversial(True)
    adv.compile(loss=['binary_crossentropy']*n_sensitive, optimizer=Adam())
    return adv

In [18]:
#Pretraining phase
def pretrain(x, y, z, classifier, adversial, epochs=10, verbose=0):
    print('Pretraining phase..')  
    trainable_classifier = _make_trainable(classifier)
    trainable_adversial = _make_trainable(adversial)
    trainable_classifier(True)
    class_weight_clf = compute_class_weights(y)
    classifier.fit(x.values, y.values, epochs=epochs, verbose=verbose, class_weight=class_weight_clf[0])
    trainable_classifier(False)
    trainable_adversial(True)
    class_weight_adv = compute_class_weights(z)
    try:
        adversial.fit(x.values, np.hsplit(z.values, z.shape[1]), class_weight=class_weight_adv[0], 
                      epochs=epochs, verbose=verbose)
    except:
        return

In [19]:

#Learning phase
def fit(x, y, z, classifier, adversial, can, validation_data=None, epochs=250, batch_size=128, save_figs=False, stop_condition=80, plot_dist=False):
    obj = 0
    print("learning phase..")
    n_sensitive = z.shape[1]

    if validation_data is not None:
        x_val, y_val, z_val = validation_data

    class_weight_adv = compute_class_weights(z)
    class_weight_clf = compute_class_weights(y)
    class_weight_can = class_weight_clf + class_weight_adv

    val_metrics = pd.DataFrame()
    fairness_metrics = pd.DataFrame()  
    delta_f_0 = pd.DataFrame()
    delta_f_1 = pd.DataFrame()
    delta_a = pd.DataFrame()
    
    trainable_classifier = _make_trainable(classifier)
    trainable_adversial = _make_trainable(adversial)

    for idx in range(epochs):

        if validation_data is not None:

            y_pred = can.predict(x_val)[0]
            val_metrics.loc[idx, 'ROC AUC'] = roc_auc_score(y_val, y_pred)
            val_metrics.loc[idx, 'Accuracy'] = (accuracy_score(y_val, (y_pred>0.5)))
            delta_a.loc[idx, "Accuracy"] =  val_metrics.loc[idx]['Accuracy']- val_metrics.loc[0]['Accuracy']


            for sensitive_attr in z_val.columns:
                fairness_metrics.loc[idx, sensitive_attr] = p_rule(y_pred, z_val[sensitive_attr])  
                
            delta_f_0.loc[idx, "Gender"] =  (fairness_metrics.loc[idx]['Gender']/100) - (fairness_metrics.loc[0]['Gender']/100)
            delta_f_1.loc[idx, "BAMEyn"] =  (fairness_metrics.loc[idx]['BAMEyn']/100) - (fairness_metrics.loc[0]['BAMEyn']/100)

            if plot_dist:
                display.clear_output(wait=True)
                
                plot_distributions(y_pred, z_val, idx+1, val_metrics.loc[idx],
                                  fairness_metrics.loc[idx],
                                  val_metrics_0=val_metrics.loc[0], p_rules_0=fairness_metrics.loc[0],
                                  fname=f'output{idx+1:08d}.png' if save_figs else None)

                plt.show(plt.gcf())
            
        # train adverserial
        trainable_classifier(False)
        trainable_adversial(True)
        try:
            h_a = adversial.fit(x.values, np.hsplit(z.values, z.shape[1]), batch_size=batch_size, epochs=1, class_weight=class_weight_adv[0], 
                                verbose=0)
        except:
            continue

        # train classifier
        trainable_classifier(True)
        trainable_adversial(False)
        indices = np.random.permutation(len(x))[:batch_size]
        h_c = can.train_on_batch(x.values[indices],
                                 [y.values[indices]]+np.hsplit(z.values[indices],n_sensitive), class_weight=class_weight_can[0])

        obj = -(((delta_f_0["Gender"][idx]/ (abs(delta_a["Accuracy"][idx])))/100) + ((delta_f_1["BAMEyn"][idx]/ (abs(delta_a["Accuracy"][idx])))/100))
        
        ep = idx+1
        if (fairness_metrics["Gender"][idx] > stop_condition and fairness_metrics["BAMEyn"][idx] > stop_condition):
                print('Number of Epochs needed to obtain fairness:', ep)
                print('delta p-rule race:', delta_f_0["Gender"][idx])
                print('delta p-rule sex:', delta_f_1["BAMEyn"][idx])
                print('delta accuracy:', delta_a["Accuracy"][idx])

                return obj
    
    print('Fairness not reached in 300 epochs:')
    return obj

In [20]:
#returns the fitting
def can(cfg):
    print("Testing configuration: \n     lambda 0:", cfg['lambda_0'], "   lambda 1: ",cfg['lambda_1'])
    n_features=X_train.shape[1]
    n_sensitive=Z_train.shape[1]
    pre_epochs=5
    pre_verbose=0 
    val_data=(X_val, y_val, Z_val)
    epochs=300
    stop_cond = 80

    classifier_inputs = Input(shape=(n_features,), name="clf_input")
    adversial_inputs = Input(shape=(1,), name="adv_input")

    classifier = build_classifier(classifier_inputs)
    adversial = build_adversial(adversial_inputs, n_sensitive)

    classifier = compile_classifier(classifier)
    can = compile_can(classifier_inputs, classifier, adversial, cfg["lambda_0"], cfg["lambda_1"])
    adversial = compile_adversial(classifier_inputs, classifier, adversial, n_sensitive)
    
    pretrain(X_train, y_train, Z_train, classifier, adversial, verbose=pre_verbose, epochs=pre_epochs)

    fitted = fit(X_train, y_train, Z_train, classifier, adversial, can, 
        validation_data=val_data,
        epochs=epochs, stop_condition=stop_cond)
    return fitted

In [21]:
#did not have time to find hyperparameter tuning with keras.tuner for the loss
# defining results dictionary that will be populated when experiments run
cfg_ei={"lambda_0": 1,"lambda_1":  200}
import random
count = 0
while can(cfg_ei) == 0 and count < 5:
    lambda_0_lower = 1.
    lambda_0_upper = 200.
    lambda_1_lower = 1.
    lambda_1_upper = 200.
    lambda_0 = random.uniform(1,200)

    lambda_1 = random.uniform(1,200)
    cfg_ei={"lambda_0": lambda_0,"lambda_1":  lambda_1}
    count = count+1
    print(cfg_ei)
can(cfg_ei)

Testing configuration: 
     lambda 0: 1    lambda 1:  200
Pretraining phase..
learning phase..
Fairness not reached in 300 epochs:
{'lambda_0': 80.18960671362748, 'lambda_1': 41.91980075458015}
Testing configuration: 
     lambda 0: 80.18960671362748    lambda 1:  41.91980075458015
Pretraining phase..
learning phase..
Fairness not reached in 300 epochs:
{'lambda_0': 152.3710102813459, 'lambda_1': 59.60107670053188}
Testing configuration: 
     lambda 0: 152.3710102813459    lambda 1:  59.60107670053188
Pretraining phase..
learning phase..
Fairness not reached in 300 epochs:
{'lambda_0': 186.03793714525915, 'lambda_1': 90.1412542961764}
Testing configuration: 
     lambda 0: 186.03793714525915    lambda 1:  90.1412542961764
Pretraining phase..
learning phase..
Fairness not reached in 300 epochs:
{'lambda_0': 27.29066807414629, 'lambda_1': 40.27640536044522}
Testing configuration: 
     lambda 0: 27.29066807414629    lambda 1:  40.27640536044522
Pretraining phase..
learning phase..
Fair

0

In [22]:
#references
#https://medium.com/hackernoon/logistic-regression-with-tensorflow-and-keras-83d2487aed89
#ALL CODE IS HEAVILY USED AND MINORLY EDITED FOR THE GIVEN DATA
#MAJORITY OF CODE DONE BY:
#https://github.com/RaffaeleAns/Adversarial-Fair-Classifier/blob/master/code.ipynb